# Creación de subconjuntos

Descargamos primero los [metadatos CSV](https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Metadata.csv) del conjunto ISIC 2019 para seleccionar 2 subconjuntos independientes:
- Conjunto 1: 100 imágenes de entrenamiento + 10 de test
- Conjunto 2: 100 imágenes de entrenamiento + 10 de test

Luego descargamos el [archivo ZIP completo](https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Input.zip) pero *solo extraemos las 220 imágenes seleccionadas, evitando problemas de espacio en el GitHub Codespace de 10GB (~220 MB vs 9+ GB completo).

## 1. Descarga del CSV y selección de imágenes

In [7]:
import pandas as pd
import requests
from pathlib import Path

csv_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Metadata.csv"
dest_dir = Path("conjunto_de_datos")
dest_dir.mkdir(parents=True, exist_ok=True)
csv_path = dest_dir / "ISIC_2019_Training_Metadata.csv"

with requests.get(csv_url, timeout=60) as r:
    r.raise_for_status()
    csv_path.write_bytes(r.content)

# Carga el CSV
df = pd.read_csv(csv_path)
print(f"Total de imágenes en el dataset: {len(df)}")
print(f"Columnas: {list(df.columns)}\n")

# Muestra las primeras filas (Referencia a Ingeniería de Sistemas Intensivos en Datos jeje)
print("Primeras 5 filas:")
print(df.head())

Total de imágenes en el dataset: 25331
Columnas: ['image', 'age_approx', 'anatom_site_general', 'lesion_id', 'sex']

Primeras 5 filas:
          image  age_approx anatom_site_general lesion_id     sex
0  ISIC_0000000        55.0      anterior torso       NaN  female
1  ISIC_0000001        30.0      anterior torso       NaN  female
2  ISIC_0000002        60.0     upper extremity       NaN  female
3  ISIC_0000003        30.0     upper extremity       NaN    male
4  ISIC_0000004        80.0     posterior torso       NaN    male


In [8]:
# Selección de 2 conjuntos (cada uno con 100 train + 10 test)
from sklearn.model_selection import train_test_split

print(f"Columnas disponibles: {list(df.columns)}\n")

# Primero separamos 220 imágenes del dataset completo
sample_size = 220
if len(df) < sample_size:
    df_subset = df
else:
    df_subset = df.sample(n=sample_size, random_state=42)

# Primera división: 110 para conjunto 1, 110 para conjunto 2
set1_df, set2_df = train_test_split(
    df_subset,
    train_size=110,
    test_size=110,
    stratify=None,
    random_state=42
)

train1_df, test1_df = train_test_split(set1_df, train_size=100, test_size=10, random_state=42)
train2_df, test2_df = train_test_split(set2_df, train_size=100, test_size=10, random_state=43)

# Guarda los 4 subconjuntos
train1_df.to_csv(dest_dir / "train1_subset.csv", index=False)
test1_df.to_csv(dest_dir / "test1_subset.csv", index=False)
train2_df.to_csv(dest_dir / "train2_subset.csv", index=False)
test2_df.to_csv(dest_dir / "test2_subset.csv", index=False)


# Verifica que NO hay solapamiento entre ningún par de conjuntos
image_col = df.columns[0]
train1_imgs = set(train1_df[image_col])
test1_imgs = set(test1_df[image_col])
train2_imgs = set(train2_df[image_col])
test2_imgs = set(test2_df[image_col])

all_sets = [
    ("train1", train1_imgs),
    ("test1", test1_imgs),
    ("train2", train2_imgs),
    ("test2", test2_imgs)
]

has_overlap = False
for i, (name1, set1) in enumerate(all_sets):
    for name2, set2 in all_sets[i+1:]:
        overlap = set1 & set2
        if overlap:
            print(f"ERROR: {len(overlap)} imágenes repetidas entre {name1} y {name2}")
            has_overlap = True

# Lista completa de imágenes a extraer
selected_images = train1_imgs | test1_imgs | train2_imgs | test2_imgs

Columnas disponibles: ['image', 'age_approx', 'anatom_site_general', 'lesion_id', 'sex']



## 2. Descarga selectiva del ZIP (solo 220 imágenes)

In [9]:
import zipfile
import io
import tempfile

zip_url = "https://isic-archive.s3.amazonaws.com/challenges/2019/ISIC_2019_Training_Input.zip"

print("Descargando el archivo ZIP completo...")
print("(Esto puede tardar varios minutos, ~9-10 GB)\n")

# Descarga con streaming a archivo temporal
with tempfile.NamedTemporaryFile(delete=False, suffix='.zip') as tmp_file:
    tmp_path = Path(tmp_file.name)
    
    with requests.get(zip_url, stream=True, timeout=600) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        downloaded = 0
        
        for chunk in r.iter_content(chunk_size=1024*1024):  # 1 MB
            if chunk:
                tmp_file.write(chunk)
                downloaded += len(chunk)
                if total_size > 0:
                    pct = (downloaded / total_size) * 100
                    print(f"Descargado: {downloaded / (1024**3):.2f} GB / {total_size / (1024**3):.2f} GB ({pct:.1f}%)", end='\r')
    
    print("\n\n✓ Descarga completa. Extrayendo solo las 220 imágenes seleccionadas...\n")
    
    # Extrae SOLO las imágenes del subset
    extracted_count = 0
    not_found = []
    
    with zipfile.ZipFile(tmp_path, 'r') as z:
        all_files = z.namelist()
        
        for img_name in selected_images:
            # Busca el archivo en el zip (puede estar en una subcarpeta)
            matching = [f for f in all_files if img_name in f and f.endswith('.jpg')]
            
            if matching:
                for file_path in matching:
                    z.extract(file_path, dest_dir)
                    extracted_count += 1
                    if extracted_count % 20 == 0:
                        print(f"Extraídas: {extracted_count}/{len(selected_images)} imágenes", end='\r')
            else:
                not_found.append(img_name)
    
    
    if not_found:
        print(f"⚠ {len(not_found)} imágenes no encontradas en el ZIP:")
        print(not_found[:5], "..." if len(not_found) > 5 else "")

# Limpia el archivo temporal
tmp_path.unlink(missing_ok=True)

Descargando el archivo ZIP completo...
(Esto puede tardar varios minutos, ~9-10 GB)

Descargado: 9.10 GB / 9.10 GB (100.0%)

✓ Descarga completa. Extrayendo solo las 220 imágenes seleccionadas...

